# PREPROCESSING

In [23]:
import pandas as pd
import numpy as np
import ast

from glob import glob
import pickle as pkl

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.svm import SVR
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from processing.parsing import *
from processing.encodings import *

In [24]:
df_ = pd.read_parquet('processed_data/tatarabuela.parquet')
df_ = df_[['price', 'lat', 'lng', 'characteristics', 'agency', 'updated', 'timestamp', 'province']].dropna()
df_churro = freeChurro(df_)
df = pd.concat([df_.reset_index(drop=True), df_churro], axis = 1)
df.drop('characteristics', axis = 1, inplace = True)

In [25]:
def tryParseM2(obj):
    try:
        if 'm²' in obj:
            return float(obj[:-2])
        return float(''.join(obj.split('.')))
    except:
        return np.nan

# df['Superficie construida'].apply(lambda x : x if pd.isna(x) else tryParseM2(x))

In [26]:
df.Baños = df.Baños.astype(float)
df['Superficie construida'] = df['Superficie construida'].apply(lambda x : x if pd.isna(x) else tryParseM2(x))

df['bathrooms'] = df['Baños']
df['surface'] = df['Superficie construida']

df.drop(['Superficie construida', 'Baños'], axis=1, inplace=True)

In [27]:
df = df[['price', 'lat', 'lng', 'Habitaciones', 'Jardín', 'Gastos de comunidad', 'Antigüedad', 'Superficie útil', 'Ascensor', 'Garaje', 'Conservación', 'agency', 'updated', 'timestamp', 'surface', 'bathrooms', 'province']]

In [28]:
df

,price,lat,lng,Habitaciones,Jardín,Gastos de comunidad,Antigüedad,Superficie útil,Ascensor,Garaje,Conservación,agency,updated,timestamp,surface,bathrooms,province
0,75000.0,38.628784,-0.761397,2,NaN,NaN,NaN,72 m²,Ascensor,NaN,NaN,Inmuebles de Topbrokers,1.673392e+09,1.699187e+15,76.0,1.0,alicante
1,56000.0,38.535113,-0.821114,3,NaN,NaN,Entre 20 y 30 años,96 m²,NaN,NaN,A reformar,Inmuebles de CICLOACTIVOS S.L.,1.673997e+09,1.699187e+15,114.0,2.0,alicante
2,39800.0,38.631641,-0.860810,2,NaN,NaN,NaN,61 m²,NaN,NaN,NaN,Inmuebles de ALTAMIRA,1.673133e+09,1.699187e+15,82.0,1.0,alicante
3,119500.0,38.537183,-0.817027,5,NaN,NaN,NaN,220 m²,NaN,NaN,En buen estado,Inmuebles de Grupo Ideas Sax,1.674170e+09,1.699187e+15,220.0,3.0,alicante
4,103000.0,38.631503,-0.765994,4,NaN,NaN,NaN,150 m²,NaN,NaN,NaN,Inmuebles de Quo Real Estate,1.672701e+09,1.699187e+15,170.0,2.0,alicante
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276087,69000.0,41.599972,-1.281389,10,NaN,NaN,NaN,290 m²,NaN,1,NaN,Inmuebles de ZARADELTA,1.674947e+09,1.699336e+15,302.0,3.0,zaragoza
276088,256800.0,41.482034,-1.373726,4,NaN,NaN,NaN,215 m²,NaN,NaN,NaN,Inmuebles de GTI ZARAGOZA,1.672960e+09,1.699336e+14,236.0,6.0,zaragoza
276089,105000.0,41.578195,-1.116482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A estrenar,Inmuebles de Aliseda Inmobiliaria,1.674343e+09,1.699336e+15,76.0,NaN,zaragoza
276090,89000.0,41.483506,-1.371575,4,NaN,NaN,NaN,100 m²,NaN,NaN,NaN,Inmuebles de INSERCONS EJEA DE LOS CABALLEROS,1.673997e+09,1.699336e+15,113.0,1.0,zaragoza


In [29]:
df['rooms'] = df['Habitaciones']
df = df.drop('Habitaciones', axis = 1)

df['garden'] = df['Jardín']
df = df.drop('Jardín', axis = 1)

df['community_expenses'] = df['Gastos de comunidad']
df = df.drop('Gastos de comunidad', axis = 1)

df['age'] = df['Antigüedad']
df = df.drop('Antigüedad', axis = 1)

df['useful_surface'] = df['Superficie útil']
df = df.drop('Superficie útil', axis = 1)

df['elevator'] = df['Ascensor']
df = df.drop('Ascensor', axis = 1)

df['garage'] = df['Garaje']
df = df.drop('Garaje', axis = 1)

df['state'] = df['Conservación']
df = df.drop('Conservación', axis = 1)

In [30]:
df = df.drop(['agency', 'updated', 'timestamp', 'community_expenses'], axis = 1)

In [31]:
df.rooms = df.rooms.astype(float)

In [32]:
df.garden = df.garden.fillna(False).apply(lambda x: True if x else False)

In [33]:
age_col = df['age'].unique()
ages = {' Menos de 5 años': 0,
                ' Entre 5 y 10 años' : 5,
                ' Entre 10 y 20 años' : 10,
                ' Entre 20 y 30 años' : 20,
                ' Entre 30 y 50 años' : 30,
                ' Más de 50 años': 50,}

In [34]:
df['age'] = df['age'].replace(ages)

In [35]:
df['useful_surface'] = df['useful_surface'].apply(lambda x : x if pd.isna(x) else tryParseM2(x))

In [36]:
df.elevator = df.elevator.fillna(False).apply(lambda x: True if x else False)

In [37]:
df.garage = df.garage.fillna(False).apply(lambda x: True if x else False)

In [38]:
# df_sample = df.sample(round(df.shape[0]*0.15), random_state=42)
# df_ = df.reset_index()[~df.reset_index()['index'].isin(df_sample.reset_index()['index'])].drop(['index'], axis=1)

In [39]:
import json

with open('../province_translate.json') as file:
    province_translate = json.load(file)

df['province'] = df.province.replace(province_translate)

In [40]:
criba = []
total_provinces = len(df.province.unique())
for i in range(1,20_000,1000):
    provinces = [province for province in df.province.unique() if len(df[df['province'] == province]) > i]
    percentage = 1 - df[df['province'].isin(provinces)].shape[0] / df.shape[0]
    n_models = len(provinces)/total_provinces
    criba.append(tuple([percentage, n_models]))

In [41]:
import plotly.express as px
px.line(criba)

In [42]:
provinces_to_train = [province for province in df.province.unique() if len(df[df['province'] == province]) > 5000]

In [44]:
len(provinces_to_train)

16

In [43]:
for province in provinces_to_train:
    df_province = df[df['province'] == province].drop('province', axis = 1)
    df_province.to_parquet(f'./processed_data/provinces/data_{province}.parquet')

In [45]:
df[~df['province'].isin(provinces_to_train)].to_parquet(f'./processed_data/provinces/data_30.parquet')

In [47]:
df_30 = pd.read_parquet('./processed_data/provinces/data_30.parquet')
df_30

,price,lat,lng,surface,bathrooms,province,rooms,garden,age,useful_surface,elevator,garage,state
104110,385000.0,42.834200,-2.788629,160.0,3.0,Araba,4.0,False,10.0,150.0,False,False,En buen estado
104111,46100.0,42.672399,-2.839510,426.0,2.0,Araba,4.0,False,50.0,293.0,False,False,None
104112,155000.0,42.875177,-3.147517,400.0,1.0,Araba,5.0,True,50.0,300.0,False,False,A reformar
104113,850000.0,42.801684,-2.898858,370.0,4.0,Araba,6.0,True,10.0,360.0,False,True,None
104114,450000.0,42.728258,-2.860074,350.0,3.0,Araba,4.0,False,NaN,300.0,False,True,En buen estado
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276087,69000.0,41.599972,-1.281389,302.0,3.0,Zaragoza,10.0,False,NaN,290.0,False,True,None
276088,256800.0,41.482034,-1.373726,236.0,6.0,Zaragoza,4.0,False,NaN,215.0,False,False,None
276089,105000.0,41.578195,-1.116482,76.0,NaN,Zaragoza,NaN,False,NaN,NaN,False,False,A estrenar
276090,89000.0,41.483506,-1.371575,113.0,1.0,Zaragoza,4.0,False,NaN,100.0,False,False,None


# Pos-procesamiento

In [ ]:
df['age'] = df['age'].fillna(df['age'].mean()) # PROBAR IMPUTAR CON KNN

In [53]:
df.state = df.state.fillna("En buen estado")

In [54]:
df_num = df._get_numeric_data()
df_cat = df.drop(df_num.columns, axis = 1)
df_cat, encodings = targetEncoding(df_cat, df_num['price'], 'target_median')
df = pd.concat([df_num, df_cat], axis = 1)

In [56]:
df_ = outliersFilter(df = df_, min_price = 50_000, max_price = 5_000_000, max_baths = 15, max_surface = 1_000)

In [62]:
df_.dropna().shape[0]/df_.shape[0]

0.6636716520689183

# Entrenamiento y validación de los 17 modelos
- 1 modelo -> 30% resto de España (tiene que entrenarse aparte de los demás porque tiene una columna extra)
- 16 modelos -> las provincias con más de 5.000 publicaciones

In [48]:
df_journal = pd.DataFrame(columns=['file', 'stage', 'r2', 'mae', 'mse', 'with_outliers']) # registro de las métricas para revisarlas a posterior

In [49]:
with_outliers = [True, False]
files = glob('./processed_data/provinces/*.parquet') # lista de direcciones de todos los archivos para tenerlos desde el inicio
# files

In [50]:
# Entrenamos con y sin outliers usando el archivo 'data_25.parquet'

for boolean in with_outliers:
  df_30 = pd.read_parquet(files[0])
  file_name = files[0].split('\\')[-1]

  df_30_validation = getSample(df_30, 0.15)
  df_30_train = df_30.reset_index()[~df_30.reset_index()['index'].isin(df_30_validation.reset_index()['index'])].drop(['index'], axis=1)

  if boolean == False:
      df_30_train = outliersFilter(df = df_30_train, min_price = 50_000, max_price = 5_000_000, max_baths = 15, max_surface = 1_000)

  df_30_cat, encodings = frequencyEncoding(df_30_train[['province', 'state']])
  df_30_train = pd.concat([df_30_train._get_numeric_data(), df_30_cat], axis = 1)

  df_30_train = df_30_train[~df_30_train['price'].isna()].dropna()

  X = df_30_train.drop(['price'], axis = 1)
  y = df_30_train[['price']]

  imputer = KNNImputer(n_neighbors=3)
  X = imputer.fit_transform(X)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
  model = RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators = 100)
  model.fit(X_train, y_train)
  yhat = model.predict(X_test)
  df_journal = pd.concat([df_journal, pd.DataFrame([[file_name,
                                                    'train',
                                                    r2_score(y_test, yhat),
                                                    mean_absolute_error(y_test, yhat),
                                                    mean_squared_error(y_test, yhat),
                                                    boolean]],
                                                  columns = ['file', 'stage', 'r2', 'mae', 'mse', 'with_outliers'])])
  
  df_30_validation['state'] = df_30_validation['state'].replace(encodings['state'])
  df_30_validation['province'] = df_30_validation['province'].replace(encodings['province'])
  df_30_validation = df_30_validation.dropna()

  X_val = df_30_validation.drop(['price'], axis = 1)
  y_val = df_30_validation[['price']]

  yhat = model.predict(X_val)

  df_journal = pd.concat([df_journal, pd.DataFrame([[file_name,
                                                    'validation',
                                                    r2_score(y_val, yhat),
                                                    mean_absolute_error(y_val, yhat),
                                                    mean_squared_error(y_val, yhat),
                                                    boolean]],
                                                  columns = ['file', 'stage', 'r2', 'mae', 'mse', 'with_outliers'])])
  
  if boolean == True:
    with open('./models/model_30.pkl', 'bw') as file:
      pkl.dump(model, file)
    with open('./models/model_30_encodings.pkl', 'bw') as file:
      pkl.dump(encodings, file)
  else:
    with open('./models/model_30_no_outliers.pkl', 'bw') as file:
      pkl.dump(model, file)
    with open('./models/model_30_no_outliers_encodings.pkl', 'bw') as file:
      pkl.dump(encodings, file)

c:\Users\cuicuidev\Desktop\pisoscom\ml\processing\encodings.py:71: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

c:\Users\cuicuidev\Desktop\pisoscom\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

C:\Users\cuicuidev\AppData\Local\Temp\ipykernel_5768\2040488244.py:27: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

c:\Users\cuicuidev\Desktop\pisos

In [51]:
df_journal

,file,stage,r2,mae,mse,with_outliers
0,data_30.parquet,train,0.757366,59017.020316,1.551734e+10,True
0,data_30.parquet,validation,0.466674,108017.630635,2.987038e+10,True
0,data_30.parquet,train,0.705738,59444.936798,1.738513e+10,False
0,data_30.parquet,validation,0.393279,121686.476174,3.398105e+10,False


In [52]:
# Entrenamos con y sin outliers usando el resto de archivos de la carpeta 'provinces'

for boolean in with_outliers:    
    for file in files[1:]:

        df = pd.read_parquet(file)
        file_name = file.split('\\')[-1]

        df_validation = df.sample(frac=0.15)
        df_train = df.reset_index()[~df.reset_index()['index'].isin(df_validation.reset_index()['index'])].drop(['index'], axis=1)

        if boolean == False:
            df_train = outliersFilter(df = df_train, min_price = 50_000, max_price = 5_000_000, max_baths = 15, max_surface = 1_000)

        df_cat, encodings = frequencyEncoding(df_train[['state']])
        df_train = pd.concat([df_train._get_numeric_data(), df_cat], axis = 1)

        df_train = df_train[~df_train['price'].isna()].dropna()

        X = df_train.drop(['price'], axis = 1)
        y = df_train[['price']]

        imputer = KNNImputer(n_neighbors=3)
        X = imputer.fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        model = RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators = 100)
        model.fit(X_train, y_train)
        yhat = model.predict(X_test)
        df_journal = pd.concat([df_journal, pd.DataFrame([[file_name,
                                                        'train',
                                                        r2_score(y_test, yhat),
                                                        mean_absolute_error(y_test, yhat),
                                                        mean_squared_error(y_test, yhat),
                                                        boolean]],
                                                        columns = ['file', 'stage', 'r2', 'mae', 'mse', 'with_outliers'])])
        
        df_validation['state'] = df_validation['state'].replace(encodings['state'])
        df_validation = df_validation.dropna()

        X_val = df_validation.drop(['price'], axis = 1)
        y_val = df_validation[['price']]

        yhat = model.predict(X_val)

        df_journal = pd.concat([df_journal, pd.DataFrame([[file_name,
                                                        'validation',
                                                        r2_score(y_val, yhat),
                                                        mean_absolute_error(y_val, yhat),
                                                        mean_squared_error(y_val, yhat),
                                                        boolean]],
                                                        columns = ['file', 'stage', 'r2', 'mae', 'mse', 'with_outliers'])])
        
        model_name = file_name.split('.')[0][5:]
        
        if boolean == True:
            with open(f'./models/model_{model_name}.pkl', 'bw') as file:
                pkl.dump(model, file)
            with open(f'./models/model_{model_name}_encodings.pkl', 'bw') as file:
                pkl.dump(encodings, file)
        else:
            with open(f'./models/model_{model_name}_no_outliers.pkl', 'bw') as file:
                pkl.dump(model, file)
            with open(f'./models/model_{model_name}_no_outliers_encodings.pkl', 'bw') as file:
                pkl.dump(encodings, file)

c:\Users\cuicuidev\Desktop\pisoscom\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\cuicuidev\Desktop\pisoscom\venv\lib\site-packages\sklearn\base.py:458: UserWarning:

X has feature names, but RandomForestRegressor was fitted without feature names

c:\Users\cuicuidev\Desktop\pisoscom\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\cuicuidev\Desktop\pisoscom\venv\lib\site-packages\sklearn\base.py:458: UserWarning:

X has feature names, but RandomForestRegressor was fitted without feature names

c:\Users\cuicuidev\Desktop\pisoscom\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please cha

In [53]:
df_journal = df_journal.reset_index(drop=True)
df_journal

,file,stage,r2,mae,mse,with_outliers
0,data_30.parquet,train,0.757366,59017.020316,1.551734e+10,True
1,data_30.parquet,validation,0.466674,108017.630635,2.987038e+10,True
2,data_30.parquet,train,0.705738,59444.936798,1.738513e+10,False
3,data_30.parquet,validation,0.393279,121686.476174,3.398105e+10,False
4,data_Alacant.parquet,train,0.718427,99393.715504,7.921714e+10,True
...,...,...,...,...,...,...
63,data_Sevilla.parquet,validation,0.688441,46358.496122,4.879038e+09,False
64,data_Tarragona.parquet,train,0.622404,57703.003972,9.618352e+09,False
65,data_Tarragona.parquet,validation,0.619216,62230.287152,1.190529e+10,False
66,data_València.parquet,train,0.749444,60495.172295,1.078191e+10,False


In [32]:
df_journal[df_journal['file']=='data_madrid.parquet']

,file,stage,r2,mae,mse,with_outliers
24,data_madrid.parquet,train,0.806927,119261.379792,8.373202e+10,True
25,data_madrid.parquet,validation,0.722236,153009.245024,1.846798e+11,True
62,data_madrid.parquet,train,0.871971,93867.218049,3.740431e+10,False
63,data_madrid.parquet,validation,0.821289,110950.715544,7.186232e+10,False


In [33]:
df_journal.to_csv('./models/journal.csv', sep=',', index=False)